# MLflow 🔄

Este notebook contiene el código usado para comparar las técnicas y los modelos generados para el TFM. 

### **Ejecutar en línea de comandos:**


cd Desktop\TFM\NEW_CONDS

mlflow ui

 http://127.0.0.1:5000

Importamos librerías:

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import time

import sklearn
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras import regularizers
# from keras import layers
# from tensorflow.keras import regularizers
# import tensorflow.keras as tk
# from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, RepeatedStratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import keras
from keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

import mlflow
# Establecer la variable de entorno GIT_PYTHON_REFRESH en quiet
os.environ["GIT_PYTHON_REFRESH"] = "quiet"

Importamos los datos:

In [2]:
df_data = pd.read_csv('data.csv')
X = df_data.drop(columns=['battery','OHS','CHS', 'chemistry'])
y = df_data['OHS']

In [3]:
X.shape, y.shape

((118, 10), (118,))

# Decision Tree 🌳

In [4]:
experiment_tree = mlflow.create_experiment("tree")

In [5]:
experiment_tree

'972786867258079225'

In [6]:
with mlflow.start_run(experiment_id=experiment_tree) as run:
    seed = 10
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    dt_classifier = DecisionTreeClassifier(random_state=seed)

    param_grid = {
        'max_depth': range(3, 11),  # Profundidad máxima del árbol
        'min_samples_split': range(2, 10),  # Número mínimo de muestras requeridas para dividir un nodo interno
        'min_samples_leaf': range(1, 8)  # Número mínimo de muestras requeridas para estar en un nodo hoja
    }
    
    mlflow.log_param('param_grid', param_grid)
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid,
                                     scoring='accuracy', cv=cv)
    
    mlflow.log_param('scoring', 'accuracy')
    mlflow.log_param('cv', cv)
    
    # Entrena el modelo
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Obtiene los mejores parámetros
    best_params = grid_search.best_params_
    print("Mejores parámetros:", best_params)

    mlflow.log_param('best_params', best_params)
    
    # Predice usando el mejor modelo encontrado
    y_train_pred = grid_search.predict(X_train)
    y_test_pred = grid_search.predict(X_test)
    
    # Calcula la precisión del conjunto de entrenamiento y de prueba
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print("Precisión del conjunto de entrenamiento:", train_accuracy)
    print("Precisión del conjunto de prueba:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)

    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    temp_csv_file = "cv_results.csv"
    cv_results_df.to_csv(temp_csv_file, index=False)

    mlflow.sklearn.log_model(grid_search, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")


C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'max_depth': 4, 'min_samples_leaf': 4, 'min_samples_split': 2}
Precisión del conjunto de entrenamiento: 0.8409090909090909
Precisión del conjunto de prueba: 0.6333333333333333
Model path: runs:/27b5c05a8a224470b142b32be2a6e19d/model


C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [28]:
experiment = mlflow.get_experiment_by_name('tree')

In [32]:
experiment.experiment_id

'972786867258079225'

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.sklearn

# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment.experiment_id) as run:
    mlflow.log_param('seed-train_test_split', 0)
    
    # Divide los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
    
    # Crea el modelo de árbol de clasificación
    mlflow.log_param('seed-DecisionTreeClassifier', 12)
    clf = DecisionTreeClassifier(random_state=12)
    
    # Entrena el modelo en los datos de entrenamiento
    clf.fit(X_train, y_train)
    
    # Realiza predicciones en los datos de prueba
    y_pred = clf.predict(X_test)
    
    # Calcula la precisión del modelo
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    
    # Registra los parámetros y métricas en MLflow
    mlflow.log_param('test_size', 0.25)
    mlflow.log_param('seed', 0)
    mlflow.log_metric('test_accuracy', accuracy)
    
    # Guarda el modelo en MLflow
    mlflow.sklearn.log_model(clf, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")


Accuracy: 0.8
Model path: runs:/751cdecd6d124593bd344900c31af917/model


In [8]:
# 

# Random Forest 🌳🌲🌴🌳

In [9]:
# experiment_rf = mlflow.create_experiment("rf")

In [15]:
experiment_rf = mlflow.get_experiment_by_name("rf")
print(experiment_rf)

<Experiment: artifact_location='file:///C:/Users/beca1/Desktop/TFM/NEW_CONDS/mlruns/153073931280205987', creation_time=1714461393030, experiment_id='153073931280205987', last_update_time=1714461393030, lifecycle_stage='active', name='rf', tags={}>


In [11]:
with mlflow.start_run(experiment_id=experiment_rf) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    rf_classifier = RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample', oob_score=True, random_state=seed)

    mlflow.log_param('bootstrap', True)
    mlflow.log_param('class_weight', 'balanced_subsample')
    mlflow.log_param('oob_score', True)
    
    param_grid = {
        'n_estimators': range(30,250),  # Número de árboles en el bosque
        'max_depth': range(3,15),  # Profundidad máxima del árbol
        'min_samples_split': range(2,10),  # Número mínimo de muestras requeridas para dividir un nodo interno
        'min_samples_leaf': range(1,10),  # Número mínimo de muestras requeridas para estar en un nodo hoja
    }
    
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    grid_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid,
                                     n_iter=100, scoring='accuracy', random_state=seed,
                                     cv=cv,) 

    mlflow.log_param('param_distributions', param_grid)
    mlflow.log_param('n_iter', 100)
    mlflow.log_param('scoring', 'accuracy')
    mlflow.log_param('cv', cv)
    
    # NOTA: SI cv=None => cv=5 por defecto.
    
    # Entrena el modelo
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Obtiene los mejores parámetros
    best_params = grid_search.best_params_
    print("Mejores parámetros:", best_params)

    mlflow.log_param('cv', cv)
    mlflow.log_param('best_params', best_params)
    
    
    # Predice usando el mejor modelo encontrado
    y_train_pred = grid_search.predict(X_train)
    y_test_pred = grid_search.predict(X_test)
    
    # Calcula la precisión del conjunto de entrenamiento y de prueba
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print("Precisión del conjunto de entrenamiento:", train_accuracy)
    print("Precisión del conjunto de prueba:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)


    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    temp_csv_file = "cv_results.csv"
    cv_results_df.to_csv(temp_csv_file, index=False)

    mlflow.sklearn.log_model(grid_search, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")

C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'n_estimators': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 7}
Precisión del conjunto de entrenamiento: 0.9886363636363636
Precisión del conjunto de prueba: 0.7
Model path: runs:/cba67deec6034af180ad9b1c6d2655ff/model


In [4]:
df_data = pd.read_csv('data.csv')
X = df_data.drop(columns=['battery','OHS','CHS'])
y = df_data['OHS']

In [5]:
X.shape, y.shape

((118, 11), (118,))

In [32]:
experiment_rf = mlflow.get_experiment_by_name("rf")
with mlflow.start_run(experiment_id=experiment_rf.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    rf_classifier = RandomForestClassifier(random_state=seed, bootstrap=True)

    # mlflow.log_param('bootstrap', True)
    # mlflow.log_param('class_weight', 'balanced_subsample')
    # mlflow.log_param('oob_score', True)
    
    param_grid = {
        'n_estimators': [50, 100, 200],  # Número de árboles en el bosque
        'max_depth': [2, 4, 6, None],  # Profundidad máxima del árbol
        'class_weight': ['balanced', 'balanced_subsample', None],
        'bootstrap': [True, False] 
    }
    
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid,
                                     scoring='accuracy', cv=cv) 

    mlflow.log_param('param_grid', param_grid)
    # mlflow.log_param('n_iter', 100)
    mlflow.log_param('scoring', 'accuracy')
    mlflow.log_param('cv', cv)
    
    # NOTA: SI cv=None => cv=5 por defecto.
    
    # Entrena el modelo
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Obtiene los mejores parámetros
    best_params = grid_search.best_params_
    print("Mejores parámetros:", best_params)

    mlflow.log_param('cv', cv)
    mlflow.log_param('best_params', best_params)
    
    
    # Predice usando el mejor modelo encontrado
    y_train_pred = grid_search.predict(X_train)
    y_test_pred = grid_search.predict(X_test)
    
    # Calcula la precisión del conjunto de entrenamiento y de prueba
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print("Precisión del conjunto de entrenamiento:", train_accuracy)
    print("Precisión del conjunto de prueba:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)


    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    temp_csv_file = "cv_results.csv"
    cv_results_df.to_csv(temp_csv_file, index=False)

    mlflow.sklearn.log_model(grid_search, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")

C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'bootstrap': True, 'class_weight': 'balanced', 'max_depth': None, 'n_estimators': 100}
Precisión del conjunto de entrenamiento: 1.0
Precisión del conjunto de prueba: 0.8
Model path: runs:/6870bebbf0934984b5b5a4b407d5f80a/model


In [38]:
experiment_rf = mlflow.get_experiment_by_name("rf")
with mlflow.start_run(experiment_id=experiment_rf.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    rf_classifier = RandomForestClassifier(bootstrap=True, 
                                           # class_weight='balanced_subsample', 
                                           oob_score=True, 
                                           random_state=seed)

    mlflow.log_param('bootstrap', True)
    # mlflow.log_param('class_weight', 'balanced_subsample')
    mlflow.log_param('oob_score', True)
    
    param_grid = {
        'n_estimators': range(30,250),  # Número de árboles en el bosque
        'max_depth': range(3,15),  # Profundidad máxima del árbol
        'min_samples_split': range(2,10),  # Número mínimo de muestras requeridas para dividir un nodo interno
        'min_samples_leaf': range(1,10),  # Número mínimo de muestras requeridas para estar en un nodo hoja
    }
    
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    grid_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid,
                                     n_iter=100, scoring='accuracy', random_state=seed,
                                     cv=cv,) 

    mlflow.log_param('param_distributions', param_grid)
    mlflow.log_param('n_iter', 100)
    mlflow.log_param('scoring', 'accuracy')
    mlflow.log_param('cv', cv)
    
    # NOTA: SI cv=None => cv=5 por defecto.
    
    # Entrena el modelo
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Obtiene los mejores parámetros
    best_params = grid_search.best_params_
    print("Mejores parámetros:", best_params)

    mlflow.log_param('cv', cv)
    mlflow.log_param('best_params', best_params)
    
    
    # Predice usando el mejor modelo encontrado
    y_train_pred = grid_search.predict(X_train)
    y_test_pred = grid_search.predict(X_test)
    
    # Calcula la precisión del conjunto de entrenamiento y de prueba
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print("Precisión del conjunto de entrenamiento:", train_accuracy)
    print("Precisión del conjunto de prueba:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)


    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    temp_csv_file = "cv_results.csv"
    cv_results_df.to_csv(temp_csv_file, index=False)

    mlflow.sklearn.log_model(grid_search, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")

C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'n_estimators': 113, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_depth': 8}
Precisión del conjunto de entrenamiento: 0.9545454545454546
Precisión del conjunto de prueba: 0.8
Model path: runs:/1126941f17b14796b8b520dff1588a24/model


In [38]:
experiment_rf = mlflow.get_experiment_by_name("rf")
with mlflow.start_run(experiment_id=experiment_rf.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    rf_classifier = RandomForestClassifier(bootstrap=True, 
                                           # class_weight='balanced_subsample', 
                                           oob_score=True, 
                                           random_state=seed)

    mlflow.log_param('bootstrap', True)
    # mlflow.log_param('class_weight', 'balanced_subsample')
    mlflow.log_param('oob_score', True)
    
    param_grid = {
        'n_estimators': range(30,250),  # Número de árboles en el bosque
        'max_depth': range(3,15),  # Profundidad máxima del árbol
        'min_samples_split': range(2,10),  # Número mínimo de muestras requeridas para dividir un nodo interno
        'min_samples_leaf': range(1,10),  # Número mínimo de muestras requeridas para estar en un nodo hoja
    }
    
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    grid_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid,
                                     n_iter=100, scoring='accuracy', random_state=seed,
                                     cv=cv,) 

    mlflow.log_param('param_distributions', param_grid)
    mlflow.log_param('n_iter', 100)
    mlflow.log_param('scoring', 'accuracy')
    mlflow.log_param('cv', cv)
    
    # NOTA: SI cv=None => cv=5 por defecto.
    
    # Entrena el modelo
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Obtiene los mejores parámetros
    best_params = grid_search.best_params_
    print("Mejores parámetros:", best_params)

    mlflow.log_param('cv', cv)
    mlflow.log_param('best_params', best_params)
    
    
    # Predice usando el mejor modelo encontrado
    y_train_pred = grid_search.predict(X_train)
    y_test_pred = grid_search.predict(X_test)
    
    # Calcula la precisión del conjunto de entrenamiento y de prueba
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print("Precisión del conjunto de entrenamiento:", train_accuracy)
    print("Precisión del conjunto de prueba:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)


    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    temp_csv_file = "cv_results.csv"
    cv_results_df.to_csv(temp_csv_file, index=False)

    mlflow.sklearn.log_model(grid_search, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")

C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'n_estimators': 113, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_depth': 8}
Precisión del conjunto de entrenamiento: 0.9545454545454546
Precisión del conjunto de prueba: 0.8
Model path: runs:/1126941f17b14796b8b520dff1588a24/model


In [39]:
experiment_rf = mlflow.get_experiment_by_name("rf")
with mlflow.start_run(experiment_id=experiment_rf.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    rf_classifier = RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample', oob_score=True, random_state=seed)

    mlflow.log_param('bootstrap', True)
    mlflow.log_param('class_weight', 'balanced_subsample')
    mlflow.log_param('oob_score', True)
    
    param_grid = {
        'n_estimators': range(30,250),  # Número de árboles en el bosque
        'max_depth': range(3,15),  # Profundidad máxima del árbol
        'min_samples_split': range(2,10),  # Número mínimo de muestras requeridas para dividir un nodo interno
        'min_samples_leaf': range(1,10),  # Número mínimo de muestras requeridas para estar en un nodo hoja
    }
    
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    grid_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid,
                                     n_iter=200, scoring='accuracy', random_state=seed,
                                     cv=cv,) 

    mlflow.log_param('param_distributions', param_grid)
    mlflow.log_param('n_iter', 200)
    mlflow.log_param('scoring', 'accuracy')
    mlflow.log_param('cv', cv)
    
    # NOTA: SI cv=None => cv=5 por defecto.
    
    # Entrena el modelo
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Obtiene los mejores parámetros
    best_params = grid_search.best_params_
    print("Mejores parámetros:", best_params)

    mlflow.log_param('cv', cv)
    mlflow.log_param('best_params', best_params)
    
    
    # Predice usando el mejor modelo encontrado
    y_train_pred = grid_search.predict(X_train)
    y_test_pred = grid_search.predict(X_test)
    
    # Calcula la precisión del conjunto de entrenamiento y de prueba
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print("Precisión del conjunto de entrenamiento:", train_accuracy)
    print("Precisión del conjunto de prueba:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)


    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    temp_csv_file = "cv_results.csv"
    cv_results_df.to_csv(temp_csv_file, index=False)

    mlflow.sklearn.log_model(grid_search, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")

C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'n_estimators': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 7}
Precisión del conjunto de entrenamiento: 0.9886363636363636
Precisión del conjunto de prueba: 0.7
Model path: runs:/32079ea082df4e59adbcf1de8fd3fd29/model


In [19]:
experiment_rf = mlflow.get_experiment_by_name("rf")
with mlflow.start_run(experiment_id=experiment_rf.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    rf_classifier = RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample', oob_score=True, random_state=seed)

    mlflow.log_param('bootstrap', True)
    mlflow.log_param('class_weight', 'balanced_subsample')
    mlflow.log_param('oob_score', True)
    
    # NOTA: SI cv=None => cv=5 por defecto.
    
    # Entrena el modelo
    start_time = time.time()
    rf_classifier.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Obtiene los mejores parámetros
    # best_params = rf_classifier.best_params_
    # print("Mejores parámetros:", best_params)

    # mlflow.log_param('cv', cv)
    # mlflow.log_param('best_params', best_params)
    
    
    # Predice usando el mejor modelo encontrado
    y_train_pred = rf_classifier.predict(X_train)
    y_test_pred = rf_classifier.predict(X_test)
    
    # Calcula la precisión del conjunto de entrenamiento y de prueba
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print("Precisión del conjunto de entrenamiento:", train_accuracy)
    print("Precisión del conjunto de prueba:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)


    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    # cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    # cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    # temp_csv_file = "cv_results.csv"
    # cv_results_df.to_csv(temp_csv_file, index=False)

    mlflow.sklearn.log_model(rf_classifier, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")

Precisión del conjunto de entrenamiento: 1.0
Precisión del conjunto de prueba: 0.8333333333333334
Model path: runs:/b05516740fe34a2997d458658d6a13f2/model


In [23]:
print(rf_classifier.n_estimators, rf_classifier.max_depth, rf_classifier.min_samples_split, rf_classifier.min_samples_leaf)

100 None 2 1


In [26]:
experiment_rf = mlflow.get_experiment_by_name("rf")
with mlflow.start_run(experiment_id=experiment_rf.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    rf_classifier = RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample', oob_score=True, random_state=seed)

    mlflow.log_param('bootstrap', True)
    mlflow.log_param('class_weight', 'balanced_subsample')
    mlflow.log_param('oob_score', True)
    
    # NOTA: SI cv=None => cv=5 por defecto.

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    grid_search = GridSearchCV(estimator=rf_classifier, scoring='accuracy',
                                     cv=cv,) 
    
    # Entrena el modelo
    start_time = time.time()
    rf_classifier.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Obtiene los mejores parámetros
    # best_params = rf_classifier.best_params_
    # print("Mejores parámetros:", best_params)

    # mlflow.log_param('cv', cv)
    # mlflow.log_param('best_params', best_params)
    
    
    # Predice usando el mejor modelo encontrado
    y_train_pred = rf_classifier.predict(X_train)
    y_test_pred = rf_classifier.predict(X_test)
    
    # Calcula la precisión del conjunto de entrenamiento y de prueba
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print("Precisión del conjunto de entrenamiento:", train_accuracy)
    print("Precisión del conjunto de prueba:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)


    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    # cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    # Convierte grid_search.cv_results_ en un DataFrame de pandas
    # cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
    # Guarda el DataFrame como un archivo CSV temporal
    # temp_csv_file = "cv_results.csv"
    # cv_results_df.to_csv(temp_csv_file, index=False)

    mlflow.sklearn.log_model(rf_classifier, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")

TypeError: GridSearchCV.__init__() missing 1 required positional argument: 'param_grid'

In [23]:
print(rf_classifier.n_estimators, rf_classifier.max_depth, rf_classifier.min_samples_split, rf_classifier.min_samples_leaf)

100 None 2 1


In [12]:
# experiment_rf = mlflow.get_experiment_by_name("rf")
# with mlflow.start_run(experiment_id=experiment_rf.experiment_id) as run:
#     seed = 0
#     mlflow.log_param('seed', seed)
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

#     rf_classifier = RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample', oob_score=True, random_state=seed)

#     mlflow.log_param('bootstrap', True)
#     mlflow.log_param('class_weight', 'balanced_subsample')
#     mlflow.log_param('oob_score', True)
    
#     param_grid = {
#         'n_estimators': range(30,250),  # Número de árboles en el bosque
#         'max_depth': range(3,15),  # Profundidad máxima del árbol
#         'min_samples_split': range(2,10),  # Número mínimo de muestras requeridas para dividir un nodo interno
#         'min_samples_leaf': range(1,10),  # Número mínimo de muestras requeridas para estar en un nodo hoja
#     }
    
    
#     cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
#     grid_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid,
#                                      n_iter=100, scoring='accuracy', random_state=seed,
#                                      cv=cv,) 

#     mlflow.log_param('param_distributions', param_grid)
#     mlflow.log_param('n_iter', 100)
#     mlflow.log_param('scoring', 'accuracy')
#     mlflow.log_param('cv', cv)
    
#     # NOTA: SI cv=None => cv=5 por defecto.
    
#     # Entrena el modelo
#     start_time = time.time()
#     grid_search.fit(X_train, y_train)
#     train_time = time.time() - start_time
    
#     # Obtiene los mejores parámetros
#     best_params = grid_search.best_params_
#     print("Mejores parámetros:", best_params)

#     mlflow.log_param('cv', cv)
#     mlflow.log_param('best_params', best_params)
    
    
#     # Predice usando el mejor modelo encontrado
#     y_train_pred = grid_search.predict(X_train)
#     y_test_pred = grid_search.predict(X_test)
    
#     # Calcula la precisión del conjunto de entrenamiento y de prueba
#     train_accuracy = accuracy_score(y_train, y_train_pred)
#     test_accuracy = accuracy_score(y_test, y_test_pred)
    
#     print("Precisión del conjunto de entrenamiento:", train_accuracy)
#     print("Precisión del conjunto de prueba:", test_accuracy)

#     mlflow.log_metric('train_time', train_time)
#     mlflow.log_metric('train_accuracy', train_accuracy)
#     mlflow.log_metric('test_accuracy', test_accuracy)


#     # Convierte grid_search.cv_results_ en un DataFrame de pandas
#     cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
#     # Guarda el DataFrame como un archivo CSV temporal
#     # Convierte grid_search.cv_results_ en un DataFrame de pandas
#     cv_results_df = pd.DataFrame(grid_search.cv_results_)
    
#     # Guarda el DataFrame como un archivo CSV temporal
#     temp_csv_file = "cv_results.csv"
#     cv_results_df.to_csv(temp_csv_file, index=False)

#     mlflow.sklearn.log_model(grid_search, 'model')
    
#     print(f"Model path: runs:/{run.info.run_id}/model")

# $K$-NN 🏡🏡🏡

In [13]:
experiment_knn = mlflow.create_experiment("knn")

In [14]:
experiment_knn

'986445357258254764'

In [15]:
with mlflow.start_run(experiment_id=experiment_knn) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)
    
    # Define el pipeline con el escalado de características y el clasificador KNN
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Escalado de características
        ('knn', KNeighborsClassifier())  # Clasificador KNN
    ])
    
    # Definir la cuadrícula de parámetros que quieres probar
    param_grid = {
        'knn__n_neighbors': [3, 4, 5, 6, 7, 8, 9],  # Prueba diferentes valores para el número de vecinos
        'knn__weights': ['uniform', 'distance'],  # Prueba diferentes métodos de ponderación
        'knn__metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev']  # Prueba diferentes métricas de distancia
    }

    mlflow.log_param('param_grid', param_grid)
    
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    mlflow.log_param('cv', cv)
    
    # Instanciar el objeto GridSearchCV
    grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='accuracy')

    mlflow.log_param('scoring', 'accuracy')
    
    # Ajustar el objeto GridSearchCV a los datos de entrenamiento
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Mostrar los mejores parámetros encontrados
    print("Mejores parámetros:", grid_search.best_params_)

    mlflow.log_param('best_params_', grid_search.best_params_)
    
    # Obtener el mejor modelo
    best_knn = grid_search.best_estimator_    
    
    # Hacer predicciones en el conjunto de prueba utilizando el mejor modelo
    y_train_pred = best_knn.predict(X_train)
    y_test_pred = best_knn.predict(X_test)
    
    # Calcular la precisión del modelo
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print("Precisión con los mejores parámetros:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)

    mlflow.sklearn.log_model(best_knn, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")


C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'knn__metric': 'euclidean', 'knn__n_neighbors': 9, 'knn__weights': 'distance'}
Precisión con los mejores parámetros: 0.8333333333333334
Model path: runs:/2a2c2ea8b8184d2c932dd83f8ccddf28/model


In [16]:
experiment_knn = mlflow.get_experiment_by_name("knn")
with mlflow.start_run(experiment_id=experiment_knn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)
    # Definir la cuadrícula de parámetros que quieres probar
    param_grid = {
        'n_neighbors': [3, 4, 5, 6, 7, 8, 9],  # Prueba diferentes valores para el número de vecinos
        'weights': ['uniform', 'distance'],  # Prueba diferentes métodos de ponderación
        'metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev']  # Prueba diferentes métricas de distancia
    }

    mlflow.log_param('param_grid', param_grid)
    
    # Crear un clasificador KNN
    knn = KNeighborsClassifier()

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    mlflow.log_param('cv', cv)
    
    # Instanciar el objeto GridSearchCV
    grid_search = GridSearchCV(knn, param_grid, cv=cv, scoring='accuracy', )

    mlflow.log_param('scoring', 'accuracy')
    
    # Ajustar el objeto GridSearchCV a los datos de entrenamiento
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Mostrar los mejores parámetros encontrados
    print("Mejores parámetros:", grid_search.best_params_)

    mlflow.log_param('best_params_', grid_search.best_params_)
    
    # Obtener el mejor modelo
    best_knn = grid_search.best_estimator_    
    
    # Hacer predicciones en el conjunto de prueba utilizando el mejor modelo
    y_train_pred = best_knn.predict(X_train)
    y_test_pred = best_knn.predict(X_test)
    
    # Calcular la precisión del modelo
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print("Precisión con los mejores parámetros:", test_accuracy)

    mlflow.log_metric('train_time', train_time)
    mlflow.log_metric('train_accuracy', train_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)

    mlflow.sklearn.log_model(best_knn, 'model')
    
    print(f"Model path: runs:/{run.info.run_id}/model")


# NN 🧬

In [17]:
# experiment_nn = mlflow.create_experiment("nn")

In [6]:
experiment_nn = mlflow.get_experiment_by_name("nn")

In [7]:
experiment_nn

<Experiment: artifact_location='file:///C:/Users/beca1/Desktop/TFM/NEW_CONDS/mlruns/691134205547675520', creation_time=1714461740776, experiment_id='691134205547675520', last_update_time=1714461740776, lifecycle_stage='active', name='nn', tags={}>

In [20]:
# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal
    model = Sequential()
    model.add(Dense(64, input_dim=10, activation='relu'))  # Capa de entrada con 10 variables de entrada
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=60, batch_size=8, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 8)
    mlflow.log_param('epochs', 50)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    mlflow.keras.log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")




Epoch 1/60


12/12 [==============================] - 2s 22ms/step - loss: 175.9107 - accuracy: 0.0638 - val_loss: 91.0260 - val_accuracy: 0.0000e+00
Epoch 2/60
12/12 [==============================] - 0s 4ms/step - loss: 67.1391 - accuracy: 0.0957 - val_loss: 44.6930 - val_accuracy: 0.0000e+00
Epoch 3/60
12/12 [==============================] - 0s 3ms/step - loss: 24.4411 - accuracy: 0.1809 - val_loss: 9.1661 - val_accuracy: 0.2083
Epoch 4/60
12/12 [==============================] - 0s 3ms/step - loss: 8.7702 - accuracy: 0.1596 - val_loss: 5.6010 - val_accuracy: 0.1667
Epoch 5/60
12/12 [==============================] - 0s 2ms/step - loss: 5.2080 - accuracy: 0.2447 - val_loss: 5.3554 - val_accuracy: 0.2500
Epoch 6/60
12/12 [==============================] - 0s 3ms/step - loss: 7.0056 - accuracy: 0.2766 - val_loss: 4.4689 - val_accuracy: 0.5417
Epoch 7/60
12/12 [==============================] - 0s 5ms/step - loss: 6.6582 - accuracy: 0.3723 - val_loss: 3.2057 - val_accuracy: 0.3333
E

2024/04/30 09:22:26 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.5416666865348816
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpbw6fv32u\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpbw6fv32u\model\data\model\assets


Model path: runs:/0bb40c688d294ce0ae5e1c0ca0823919/model


In [21]:
# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal
    model = Sequential()
    model.add(Dense(64, input_dim=10, activation='relu'))  # Capa de entrada con 10 variables de entrada
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 8)
    mlflow.log_param('epochs', 50)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    mlflow.keras.log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/50
6/6 [==============================] - 1s 46ms/step - loss: 796.9472 - accuracy: 0.0106 - val_loss: 407.5957 - val_accuracy: 0.0000e+00
Epoch 2/50
6/6 [==============================] - 0s 7ms/step - loss: 300.9098 - accuracy: 0.0426 - val_loss: 119.6068 - val_accuracy: 0.1250
Epoch 3/50
6/6 [==============================] - 0s 8ms/step - loss: 173.0491 - accuracy: 0.1064 - val_loss: 88.8016 - val_accuracy: 0.1667
Epoch 4/50
6/6 [==============================] - 0s 5ms/step - loss: 122.7944 - accuracy: 0.2128 - val_loss: 17.5652 - val_accuracy: 0.2500
Epoch 5/50
6/6 [==============================] - 0s 14ms/step - loss: 89.8905 - accuracy: 0.3085 - val_loss: 28.3147 - val_accuracy: 0.3750
Epoch 6/50
6/6 [==============================] - 0s 7ms/step - loss: 62.9286 - accuracy: 0.1702 - val_loss: 21.2663 - val_accuracy: 0.1667
Epoch 7/50
6/6 [==============================] - 0s 7ms/step - loss: 44.0135 - accuracy: 0.2021 - val_loss: 11.9712 - val_accuracy: 0.0833
Epoch 8/

2024/04/30 09:22:43 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.5416666865348816
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpcs0919iw\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpcs0919iw\model\data\model\assets


Model path: runs:/a45707f21bdb47f0b622d63dce2499e3/model


In [22]:
# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal
    model = Sequential()
    model.add(Dense(64, input_dim=10, activation='relu'))  # Capa de entrada con 10 variables de entrada
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 8)
    mlflow.log_param('epochs', 50)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    mlflow.keras.log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/100
3/3 [==============================] - 1s 214ms/step - loss: 382.2212 - accuracy: 0.1383 - val_loss: 296.2609 - val_accuracy: 0.1250
Epoch 2/100
3/3 [==============================] - 0s 15ms/step - loss: 246.4609 - accuracy: 0.0957 - val_loss: 234.3517 - val_accuracy: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 0s 16ms/step - loss: 179.5833 - accuracy: 0.0745 - val_loss: 179.9067 - val_accuracy: 0.2083
Epoch 4/100
3/3 [==============================] - 0s 15ms/step - loss: 168.2442 - accuracy: 0.1064 - val_loss: 169.8383 - val_accuracy: 0.2083
Epoch 5/100
3/3 [==============================] - 0s 14ms/step - loss: 133.5508 - accuracy: 0.1596 - val_loss: 139.5296 - val_accuracy: 0.0417
Epoch 6/100
3/3 [==============================] - 0s 17ms/step - loss: 116.8157 - accuracy: 0.1809 - val_loss: 125.7337 - val_accuracy: 0.1667
Epoch 7/100
3/3 [==============================] - 0s 24ms/step - loss: 90.4573 - accuracy: 0.2128 - val_loss: 98.8148 - val_accura

2024/04/30 09:22:59 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.5833333134651184
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpkf556oko\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpkf556oko\model\data\model\assets


Model path: runs:/b898440e48584b4f8c69f5aacedb8f74/model


In [23]:
# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal
    model = Sequential()
    model.add(Dense(64, input_dim=10, activation='relu'))  # Capa de entrada con 10 variables de entrada
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=100, batch_size=24, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 8)
    mlflow.log_param('epochs', 50)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    mlflow.keras.log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/100
4/4 [==============================] - 1s 73ms/step - loss: 238.9168 - accuracy: 0.1383 - val_loss: 113.6974 - val_accuracy: 0.2083
Epoch 2/100
4/4 [==============================] - 0s 9ms/step - loss: 134.2016 - accuracy: 0.1277 - val_loss: 102.4957 - val_accuracy: 0.0417
Epoch 3/100
4/4 [==============================] - 0s 12ms/step - loss: 89.2926 - accuracy: 0.1489 - val_loss: 29.8266 - val_accuracy: 0.1250
Epoch 4/100
4/4 [==============================] - 0s 8ms/step - loss: 64.1651 - accuracy: 0.1383 - val_loss: 34.0963 - val_accuracy: 0.2083
Epoch 5/100
4/4 [==============================] - 0s 13ms/step - loss: 68.2033 - accuracy: 0.1702 - val_loss: 32.1906 - val_accuracy: 0.1667
Epoch 6/100
4/4 [==============================] - 0s 8ms/step - loss: 72.6701 - accuracy: 0.1170 - val_loss: 11.3316 - val_accuracy: 0.1250
Epoch 7/100
4/4 [==============================] - 0s 21ms/step - loss: 52.2877 - accuracy: 0.2128 - val_loss: 36.1696 - val_accuracy: 0.1667
Epoch

2024/04/30 09:23:15 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.625
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmp8w0yqm84\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmp8w0yqm84\model\data\model\assets


Model path: runs:/df61aa8d5d47456c875cdea074bd374f/model


In [9]:
import mlflow
from mlflow.keras import log_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
import time

# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal convolucional
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(10, 1)))  # Capa convolucional
    model.add(MaxPooling1D(2))  # Capa de pooling
    model.add(Flatten())  # Capa de aplanamiento
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=100, batch_size=24, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 24)
    mlflow.log_param('epochs', 100)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")





Epoch 1/100


4/4 [==============================] - 8s 80ms/step - loss: 92.0115 - accuracy: 0.1383 - val_loss: 93.3998 - val_accuracy: 0.2500
Epoch 2/100
4/4 [==============================] - 0s 14ms/step - loss: 58.2114 - accuracy: 0.1383 - val_loss: 73.4605 - val_accuracy: 0.1667
Epoch 3/100
4/4 [==============================] - 0s 9ms/step - loss: 49.2975 - accuracy: 0.1383 - val_loss: 71.2658 - val_accuracy: 0.0417
Epoch 4/100
4/4 [==============================] - 0s 14ms/step - loss: 39.2207 - accuracy: 0.1277 - val_loss: 55.6354 - val_accuracy: 0.1250
Epoch 5/100
4/4 [==============================] - 0s 11ms/step - loss: 28.3136 - accuracy: 0.1596 - val_loss: 41.7976 - val_accuracy: 0.4167
Epoch 6/100
4/4 [==============================] - 0s 10ms/step - loss: 23.2101 - accuracy: 0.3617 - val_loss: 36.0172 - val_accuracy: 0.4167
Epoch 7/100
4/4 [==============================] - 0s 20ms/step - loss: 19.8160 - accuracy: 0.2234 - val_loss: 35.6960 - val_accuracy: 0.2083
Ep

2024/05/08 10:27:21 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.6666666865348816
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpkhpgfnbr\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpkhpgfnbr\model\data\model\assets


Model path: runs:/60ce056e81454ae2ad4a637428177173/model


C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [9]:
import mlflow
from mlflow.keras import log_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
import time

# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal convolucional
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(10, 1)))  # Capa convolucional
    model.add(MaxPooling1D(2))  # Capa de pooling
    model.add(Flatten())  # Capa de aplanamiento
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=100, batch_size=24, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 24)
    mlflow.log_param('epochs', 100)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")





Epoch 1/100


4/4 [==============================] - 8s 80ms/step - loss: 92.0115 - accuracy: 0.1383 - val_loss: 93.3998 - val_accuracy: 0.2500
Epoch 2/100
4/4 [==============================] - 0s 14ms/step - loss: 58.2114 - accuracy: 0.1383 - val_loss: 73.4605 - val_accuracy: 0.1667
Epoch 3/100
4/4 [==============================] - 0s 9ms/step - loss: 49.2975 - accuracy: 0.1383 - val_loss: 71.2658 - val_accuracy: 0.0417
Epoch 4/100
4/4 [==============================] - 0s 14ms/step - loss: 39.2207 - accuracy: 0.1277 - val_loss: 55.6354 - val_accuracy: 0.1250
Epoch 5/100
4/4 [==============================] - 0s 11ms/step - loss: 28.3136 - accuracy: 0.1596 - val_loss: 41.7976 - val_accuracy: 0.4167
Epoch 6/100
4/4 [==============================] - 0s 10ms/step - loss: 23.2101 - accuracy: 0.3617 - val_loss: 36.0172 - val_accuracy: 0.4167
Epoch 7/100
4/4 [==============================] - 0s 20ms/step - loss: 19.8160 - accuracy: 0.2234 - val_loss: 35.6960 - val_accuracy: 0.2083
Ep

2024/05/08 10:27:21 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.6666666865348816
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpkhpgfnbr\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpkhpgfnbr\model\data\model\assets


Model path: runs:/60ce056e81454ae2ad4a637428177173/model


C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [12]:
import mlflow
from mlflow.keras import log_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
import time

# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal convolucional
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(10, 1)))  # Capa convolucional
    model.add(MaxPooling1D(2))  # Capa de pooling
    model.add(Flatten())  # Capa de aplanamiento
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=200, batch_size=24, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 24)
    mlflow.log_param('epochs', 100)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/200
4/4 [==============================] - 1s 68ms/step - loss: 201.6560 - accuracy: 0.0532 - val_loss: 89.6724 - val_accuracy: 0.1250
Epoch 2/200
4/4 [==============================] - 0s 12ms/step - loss: 126.6375 - accuracy: 0.1596 - val_loss: 60.0890 - val_accuracy: 0.0000e+00
Epoch 3/200
4/4 [==============================] - 0s 17ms/step - loss: 83.0044 - accuracy: 0.1170 - val_loss: 39.9948 - val_accuracy: 0.1667
Epoch 4/200
4/4 [==============================] - 0s 26ms/step - loss: 57.9562 - accuracy: 0.2234 - val_loss: 20.1952 - val_accuracy: 0.4167
Epoch 5/200
4/4 [==============================] - 0s 10ms/step - loss: 37.5848 - accuracy: 0.3723 - val_loss: 23.6413 - val_accuracy: 0.4167
Epoch 6/200
4/4 [==============================] - 0s 9ms/step - loss: 34.0672 - accuracy: 0.3617 - val_loss: 15.5770 - val_accuracy: 0.4167
Epoch 7/200
4/4 [==============================] - 0s 11ms/step - loss: 23.4407 - accuracy: 0.4043 - val_loss: 13.3430 - val_accuracy: 0.3333
E

2024/05/08 10:30:26 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.7916666865348816
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpob3lts1u\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpob3lts1u\model\data\model\assets


Model path: runs:/6e6c27a6c51c4665b2a04aeb9d77a715/model


In [13]:
import mlflow
from mlflow.keras import log_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
import time

# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal convolucional
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(10, 1)))  # Capa convolucional
    model.add(MaxPooling1D(2))  # Capa de pooling
    model.add(Flatten())  # Capa de aplanamiento
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=500, batch_size=24, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 24)
    mlflow.log_param('epochs', 100)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/500
4/4 [==============================] - 1s 75ms/step - loss: 91.2464 - accuracy: 0.0851 - val_loss: 17.3760 - val_accuracy: 0.2083
Epoch 2/500
4/4 [==============================] - 0s 16ms/step - loss: 43.9216 - accuracy: 0.1383 - val_loss: 24.9023 - val_accuracy: 0.1667
Epoch 3/500
4/4 [==============================] - 0s 23ms/step - loss: 37.0291 - accuracy: 0.2340 - val_loss: 21.3311 - val_accuracy: 0.2083
Epoch 4/500
4/4 [==============================] - 0s 14ms/step - loss: 17.5399 - accuracy: 0.0851 - val_loss: 24.6320 - val_accuracy: 0.0417
Epoch 5/500
4/4 [==============================] - 0s 11ms/step - loss: 16.3822 - accuracy: 0.1809 - val_loss: 24.4107 - val_accuracy: 0.2500
Epoch 6/500
4/4 [==============================] - 0s 10ms/step - loss: 15.6106 - accuracy: 0.3723 - val_loss: 23.2466 - val_accuracy: 0.2083
Epoch 7/500
4/4 [==============================] - 0s 11ms/step - loss: 12.5763 - accuracy: 0.2979 - val_loss: 12.9568 - val_accuracy: 0.5000
Epoch 

2024/05/08 10:31:51 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.7916666865348816
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmphv89n2zh\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmphv89n2zh\model\data\model\assets


Model path: runs:/21e0a844c523470dbb479279891f1b81/model


In [14]:
import mlflow
from mlflow.keras import log_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
import time

# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal convolucional
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(10, 1)))  # Capa convolucional
    model.add(MaxPooling1D(2))  # Capa de pooling
    model.add(Flatten())  # Capa de aplanamiento
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=1000, batch_size=24, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 24)
    mlflow.log_param('epochs', 100)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/1000
4/4 [==============================] - 1s 76ms/step - loss: 185.0198 - accuracy: 0.0638 - val_loss: 141.1467 - val_accuracy: 0.1250
Epoch 2/1000
4/4 [==============================] - 0s 29ms/step - loss: 130.6278 - accuracy: 0.1489 - val_loss: 96.7244 - val_accuracy: 0.1250
Epoch 3/1000
4/4 [==============================] - 0s 13ms/step - loss: 85.1191 - accuracy: 0.1489 - val_loss: 46.7155 - val_accuracy: 0.0833
Epoch 4/1000
4/4 [==============================] - 0s 11ms/step - loss: 57.6495 - accuracy: 0.2340 - val_loss: 42.9108 - val_accuracy: 0.4167
Epoch 5/1000
4/4 [==============================] - 0s 11ms/step - loss: 57.5951 - accuracy: 0.3511 - val_loss: 31.0612 - val_accuracy: 0.4167
Epoch 6/1000
4/4 [==============================] - 0s 12ms/step - loss: 39.2467 - accuracy: 0.2021 - val_loss: 21.3863 - val_accuracy: 0.1250
Epoch 7/1000
4/4 [==============================] - 0s 28ms/step - loss: 28.5090 - accuracy: 0.1170 - val_loss: 9.0172 - val_accuracy: 0.12

2024/05/08 10:38:41 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.7916666865348816
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpwmdni4jt\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpwmdni4jt\model\data\model\assets


Model path: runs:/43d5c70afff549839095646877cabdd3/model


In [11]:
import mlflow
from mlflow.keras import log_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
import time

# Inicia un nuevo run de MLflow
with mlflow.start_run(experiment_id=experiment_nn.experiment_id) as run:
    seed = 0
    mlflow.log_param('seed', seed)

    # Codifica las etiquetas a valores numéricos si aún no están codificadas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

    # Define el modelo de la red neuronal con dropout
    model = Sequential()
    model.add(Dense(64, input_dim=10, activation='relu'))  # Capa de entrada con 10 variables de entrada
    model.add(Dropout(0.5))  # Capa de dropout
    model.add(Dense(32, activation='relu'))  # Capa oculta con 32 neuronas
    model.add(Dropout(0.5))  # Capa de dropout
    model.add(Dense(8, activation='softmax'))  # Capa de salida con 8 neuronas para las 8 categorías

    # Compila el modelo
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Entrena el modelo
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=200, batch_size=24, validation_data=(X_test, y_test))
    train_time = time.time() - start_time

    # Evalúa el modelo en el conjunto de prueba
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy del modelo en el conjunto de prueba:", accuracy)
    
    # Log de los parámetros y métricas en MLflow
    mlflow.log_param('batch_size', 24)
    mlflow.log_param('epochs', 100)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el modelo en MLflow
    log_model(model, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")


Epoch 1/200
4/4 [==============================] - 1s 65ms/step - loss: 1838.0234 - accuracy: 0.1489 - val_loss: 954.7469 - val_accuracy: 0.2083
Epoch 2/200
4/4 [==============================] - 0s 7ms/step - loss: 1924.5314 - accuracy: 0.1064 - val_loss: 740.5913 - val_accuracy: 0.2083
Epoch 3/200
4/4 [==============================] - 0s 19ms/step - loss: 1436.1852 - accuracy: 0.1489 - val_loss: 540.9675 - val_accuracy: 0.2083
Epoch 4/200
4/4 [==============================] - 0s 12ms/step - loss: 1136.7094 - accuracy: 0.2021 - val_loss: 380.0342 - val_accuracy: 0.2083
Epoch 5/200
4/4 [==============================] - 0s 12ms/step - loss: 990.6725 - accuracy: 0.1277 - val_loss: 248.3946 - val_accuracy: 0.2500
Epoch 6/200
4/4 [==============================] - 0s 10ms/step - loss: 1063.8324 - accuracy: 0.1702 - val_loss: 138.5972 - val_accuracy: 0.3333
Epoch 7/200
4/4 [==============================] - 0s 10ms/step - loss: 953.5150 - accuracy: 0.1596 - val_loss: 98.2181 - val_accura

2024/05/08 10:29:45 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Accuracy del modelo en el conjunto de prueba: 0.4166666567325592
INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpoaqc136j\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\beca1\AppData\Local\Temp\tmpoaqc136j\model\data\model\assets


Model path: runs:/e94f1281e6a14d5d815bf1520331450e/model


# SVM ⚙⚙⚙

In [24]:
experiment_svm = mlflow.create_experiment("svm")

In [25]:
experiment_svm

'322919668359299300'

In [26]:
# experiment_svm = mlflow.get_experiment_by_name("svm")
# with mlflow.start_run(experiment_id=experiment_svm.experiment_id) as run:
#     seed = 0
#     mlflow.log_param('seed', seed)
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

#     # Define los parámetros de búsqueda en rejilla
#     param_grid = {
#         'C': [0.01, 0.1, 1, 5, 10, 15, 20],  # Prueba diferentes valores para el parámetro de regularización C
#         'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1],  # Prueba diferentes valores para el parámetro gamma del kernel
#         'kernel': ['rbf']  # Prueba diferentes tipos de kernel
#     }

#     mlflow.log_param('param_grid', param_grid)

#     # Construye el modelo SVM
#     svm = SVC(random_state=seed)

#     cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
#     mlflow.log_param('cv', cv)
    
#     # Realiza la búsqueda en rejilla
#     grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=cv,
#                                scoring='accuracy', n_jobs=-1)

#     start_time = time.time()
#     grid_result = grid_search.fit(X_train, y_train)
#     train_time = time.time() - start_time

#     # Muestra los mejores parámetros encontrados
#     print("Mejores parámetros:", grid_result.best_params_)
#     mlflow.log_param('best_params_', grid_result.best_params_)

#     # Hacer predicciones en el conjunto de prueba utilizando el mejor modelo
#     y_test_pred = grid_result.predict(X_test)

#     # Calcular la precisión del modelo
#     test_accuracy = accuracy_score(y_test, y_test_pred)
#     print("Precisión del modelo en el conjunto de prueba:", test_accuracy)
#     mlflow.log_metric('test_accuracy', test_accuracy)
#     mlflow.log_metric('train_time', train_time)
    

#     # Guarda el mejor modelo en MLflow
#     mlflow.sklearn.log_model(grid_result.best_estimator_, "model")
#     print(f"Model path: runs:/{run.info.run_id}/model")

In [27]:
with mlflow.start_run(experiment_id=experiment_svm) as run:
    seed = 0
    mlflow.log_param('seed', seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

    # Define los parámetros de búsqueda en rejilla
    param_grid = {
        'C': [0.01, 0.1, 1, 10],  # Prueba diferentes valores para el parámetro de regularización C
        'gamma': [0.0001, 0.001, 0.01, 0.1, 1],  # Prueba diferentes valores para el parámetro gamma del kernel
        'kernel': ['linear', 'rbf']  # Prueba diferentes tipos de kernel
    }

    mlflow.log_param('param_grid', param_grid)

    # Construye el modelo SVM
    svm = SVC(random_state=seed)

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=seed)
    mlflow.log_param('cv', cv)

    # Realiza la búsqueda en rejilla
    grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=cv,
                               scoring='accuracy', n_jobs=-1)

    start_time = time.time()
    grid_result = grid_search.fit(X_train, y_train)
    train_time = time.time() - start_time

    # Muestra los mejores parámetros encontrados
    print("Mejores parámetros:", grid_result.best_params_)
    mlflow.log_param('best_params_', grid_result.best_params_)

    # Hacer predicciones en el conjunto de prueba utilizando el mejor modelo
    y_test_pred = grid_result.predict(X_test)

    # Calcular la precisión del modelo
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print("Precisión del modelo en el conjunto de prueba:", test_accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)
    mlflow.log_metric('train_time', train_time)
    

    # Guarda el mejor modelo en MLflow
    mlflow.sklearn.log_model(grid_result.best_estimator_, "model")
    print(f"Model path: runs:/{run.info.run_id}/model")

C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\beca1\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Mejores parámetros: {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
Precisión del modelo en el conjunto de prueba: 0.7666666666666667
Model path: runs:/e7b03dc62dc44be389f19845cf7d51e0/model
